In [ ]:
import pymysql
wikiconn  = pymysql.connect(host='localhost',
                            user='',
                            password='',
                            port=4711,
                            db='enwiki_p',
                            charset='utf8')
wikiCur = wikiconn.cursor()

In [3]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i+n]

['x', 'x', 'x', 'x', 'x']

In [ ]:
import datetime
import pandas as pd

def PrintableTimestamp():
    now = datetime.datetime.now().time()
    return "[{:02d}:{:02d}:{:02d}]".format(now.hour,now.minute,now.second)

def BatchQueryFromList(listOfIDs,columnThatContainsIDs,tableToQuery,listOfColumnsToRetrieve):
    '''listOfIDs is a list of keys to retrieve from an SQL table
    columnThatContainsIDs is the column in that table that stores the keys
    tableToQuery is the table containing the information you want
    listOfColumnsToRetrieve is a list containing the SQL DB table column names you want to retrieve
    Returns a pandas dataframe with column names equivalent to those in the DB you queried'''
    baseQ = "SELECT {} FROM {} WHERE {} IN ({})"
    colsForQ = ",".join(listOfColumnsToRetrieve)
    idChunks = chunks(listOfIDs, 1000)
    result_df = None
    for ix, chunk in enumerate(idChunks):
        print(PrintableTimestamp() + " fetching chunk {}".format(ix))
        idsInChunk = ",".join(chunk)
        preppedQ = baseQ.format(colsForQ,tabletoQuery,columnThatContainsIDs,idsInChunk)
        wikiCur.execute(preppedQ)
        if result_df:
            result_df.append(list(wikiCur.fetchall()))
        else:
            result_df = pd.DataFrame(list(wikiCur.fetchall()),columns=listOfColumnsToRetrieve)
    return result_df
    